# Clustering Crypto

In [52]:
# Initial imports
import pandas as pd
import hvplot.pandas
# from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [90]:
# Load the crypto_data.csv dataset.
df = pd.read_csv("crypto_data.csv")
df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [91]:
# Keep all the cryptocurrencies that are being traded.
trading_df = df[df['IsTrading'] == True]
trading_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [92]:
# Keep all the cryptocurrencies that have a working algorithm.
alg_df = trading_df[df['Algorithm'] != '']
alg_df.head()

c:\Users\pmerc\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [93]:
# Remove the "IsTrading" column. 
df = alg_df.drop(columns='IsTrading')
df.head()


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [94]:
# Remove rows that have at least 1 null value.
df = df.dropna()
df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [95]:
# Keep the rows where coins are mined.
df = df[df['TotalCoinsMined'] > 0]
df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [96]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_names_df = df['CoinName']
coin_names_df.reset_index(inplace = True, drop = True)
coin_names_df.head()

0      42 Coin
1      404Coin
2    EliteCoin
3      Bitcoin
4     Ethereum
Name: CoinName, dtype: object

In [97]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df.drop(columns = ['CoinName','Unnamed: 0'], inplace=True, axis = 1)


In [98]:
df.reset_index(inplace= True, drop = True)
df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
1,Scrypt,PoW/PoS,1.055185e+09,532000000
2,X13,PoW/PoS,2.927942e+10,314159265359
3,SHA-256,PoW,1.792718e+07,21000000
4,Ethash,PoW,1.076842e+08,0


In [99]:
# Use get_dummies() to create variables for text features.
dummies_df = pd.get_dummies(df, columns = ['Algorithm', 'ProofType'])
dummies_df.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
# Standardize the data with StandardScaler().
df_scaled = StandardScaler().fit_transform(dummies_df)
print(df_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [101]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components = 3)
pca_df = pca.fit_transform(df_scaled)
pca_df

array([[-0.34159475,  1.03964624, -0.56625225],
       [-0.32494526,  1.0399293 , -0.56650549],
       [ 2.30810375,  1.74924761, -0.58472119],
       ...,
       [ 0.33082846, -2.28662128,  0.38580244],
       [-0.12735337, -2.10826122,  0.42111015],
       [-0.28983835,  0.8337616 , -0.23167659]])

In [102]:
# Create a DataFrame with the three principal components.
df_pca_scaled = pd.DataFrame(data = pca_df, columns = ['PC1', 'PC2', 'PC3'])
df_pca_scaled.head()

,PC1,PC2,PC3
0,-0.341595,1.039646,-0.566252
1,-0.324945,1.039929,-0.566505
2,2.308104,1.749248,-0.584721
3,-0.144961,-1.251515,0.195276
4,-0.158976,-1.988716,0.353802


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [103]:
# Create an elbow curve to find the best value for K.
#finding best value for k
inertia = []
k = list(range(1,11))

# Calculating interia for range of K values
for i in k:
    km = KMeans(n_clusters= i, random_state = 0)
    km.fit(df_pca_scaled)
    inertia.append(km.inertia_)

#Creating the elbow curve
elbow_data = {'k': k, 'inertia': inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k', y='inertia', xticks = k, title = 'Elbow Curve')


c:\Users\pmerc\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [104]:
# Initialize the K-Means model.
model = KMeans(n_clusters= 4, random_state = 0)

# Fit the model
model.fit(df_pca_scaled)

# Predict clusters
predictions = model.predict(df_pca_scaled)

In [105]:
predictions.shape

(532,)

In [106]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, df_pca_scaled], axis =1, sort = False)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = coin_names_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,Class
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.341595,1.039646,-0.566252,42 Coin,0
1,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.324945,1.039929,-0.566505,404Coin,0
2,X13,PoW/PoS,2.927942e+10,314159265359,2.308104,1.749248,-0.584721,EliteCoin,0
3,SHA-256,PoW,1.792718e+07,21000000,-0.144961,-1.251515,0.195276,Bitcoin,3
4,Ethash,PoW,1.076842e+08,0,-0.158976,-1.988716,0.353802,Ethereum,3
5,Scrypt,PoW,6.303924e+07,84000000,-0.170705,-1.128863,-0.029348,Litecoin,3
6,X11,PoW/PoS,9.031294e+06,22000000,-0.401129,1.159409,-0.501291,Dash,0
7,CryptoNight-V7,PoW,1.720114e+07,0,-0.153490,-2.196773,0.341069,Monero,3
8,Ethash,PoW,1.133597e+08,210000000,-0.157419,-1.988804,0.353797,Ethereum Classic,3
9,Equihash,PoW,7.383056e+06,21000000,-0.127352,-2.108261,0.421110,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [128]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x = 'PC1',
    y = 'PC2',
    z = 'PC3',
    color = 'Class',
    symbol = 'Class',
    hover_name='CoinName',
    hover_data=['Algorithm']
)
fig.update_layout(
    legend=dict(x=0,y=1)
) 
fig.show()


In [130]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [131]:
# Print the total number of tradable cryptocurrencies.
total_cc = len(clustered_df)
print(f'There are {total_cc} tradeable cryptocurrencies.')

There are 532 tradeable cryptocurrencies.


In [157]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
clustered_X_scaled = MinMaxScaler().fit_transform(X=X)
clustered_X_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [158]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(clustered_X_scaled, columns = ['TotalCoinSupply','TotalCoinsMined'], index = clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df['Class']

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,4.200000e-11,0.000000,42 Coin,0
1,5.320000e-04,0.001066,404Coin,0
2,3.141593e-01,0.029576,EliteCoin,0
3,2.100000e-05,0.000018,Bitcoin,3
4,0.000000e+00,0.000109,Ethereum,3
5,8.400000e-05,0.000064,Litecoin,3
6,2.200000e-05,0.000009,Dash,0
7,0.000000e+00,0.000017,Monero,3
8,2.100000e-04,0.000115,Ethereum Classic,3
9,2.100000e-05,0.000007,ZCash,3


In [161]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x='TotalCoinsMined', y = 'TotalCoinSupply', by='Class', hover = True, hover_cols = 'CoinName', legend = True)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)